In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import confusion_matrix, classification_report,f1_score
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np
from scipy.spatial.distance import cdist
import heapq
from collections import Counter
import os
import ast
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import random
import lime
random.seed(1993)
random.seed(1993)
%matplotlib inline

In [2]:
df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
df_test

,text,spans
0,Evanukachum pondati kolandhainu sentiment irun...,"[71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 8..."
1,Adei ennada short film la irunthu suturukinga,"[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]"
2,Super dialogue Oruthar mela visvasam kattrath...,"[68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 7..."
3,Epadiya jathi padam. Ponnu padama edungada. In...,"[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7..."
4,Ponnu mella kaivaikuravan Kai mattum illa uyir...,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5..."
...,...,...
871,காலா.காபலி.அசுரன்.பாரியேரும்.பெருமள் வந்த அப்ப...,"[101, 102, 103, 104, 105, 106, 107, 108, 109, ..."
872,Kekka bekka short film mathri irukey,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]"
873,Bayangaram... Trailerey ippadina appa Padam en...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
874,komali rasini vesam pottu tamil ilichavayangal...,"[0, 1, 2, 3, 4, 5, 6, 7, 31, 32, 33, 34, 35, 3..."


In [3]:
# Complex + Multilabel

In [4]:
# Read the output and create spans

In [5]:
def f1_span(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        if len(predictions) == 0:
            return 0
        else:
            return 1
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [6]:
# Complex-Multi

In [ ]:
#BERT ONLY

import re

def eval_multiple(LIMEFILE,MAXSIZE,MINSIZE):
    
    output_xlm_cm = np.load(LIMEFILE,allow_pickle=True)
    new_output_xlm=[]
    df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
#     print(output_xlm_cm)
    start=0
    full_res=[]
    for index in range(len(output_xlm_cm)):
        modified=[]
        temp = output_xlm_cm[index]
        
#         print(temp)
        start=0
        for item in temp:
            w,s = [*item.keys()][0],[*item.values()][0]
#             print(w,s)
            if "CAUSE" in s:
                modified.append((w,1.0,[i for i in range(start,start+len(w))]))
                start+=len(w)+1
            else:
                modified.append((w,0.0,[i for i in range(start,start+len(w))]))
                start+=len(w)+1
        
        full_res.append(modified)
#         print(modified,temp,"\n\n")

#     print(full_res)
    
    THRESH=0
    
    totalscore=[]
    for index in range(len(full_res)):
        if len(df_test["text"].values.tolist()[index])>=MINSIZE and len(df_test["text"].values.tolist()[index])<=MAXSIZE:
            gt = df_test["spans"].values.tolist()[index]
            predspan=[]
#             print(full_res[index])
            for word,score,sp in full_res[index]:
                if score>THRESH:
                    predspan+=sp
            ts = [i for i in ast.literal_eval(gt)]
            sc = f1_span(predspan, ts)
            ttx = df_test["text"].values.tolist()[index]
            
            ggtext=""
            predtext=""
            
            ttx=list(ttx)
            for i in predspan:
                predtext+=ttx[i]
            
            for i in ts:
                try:
                    ggtext+=ttx[i]
                except:
                    pass
            print("".join(ttx),"|",predtext,"|",ggtext,"|",sc)
#             print("Text missing in pred:",)
#             print("Text missing in gt:")
            input()
            totalscore.append(sc)
    
    print("Results:",np.average(totalscore))
    
LIMEFILE = "../models/ner/roberta/roberta_ner.py.npy"

MAXSIZE = 10000
MINSIZE = 0
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

MAXSIZE = 30
MINSIZE = 0
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

MAXSIZE = 50
MINSIZE = 30
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

MAXSIZE = 500
MINSIZE = 50
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

Evanukachum pondati kolandhainu sentiment irundha , apdiyaeoadi poairu, kola gandula iruken mavana kollama vida mata | kolagandulairukenmavanakollamavidamata |  kola gandula iruken mavana kollama vida mata | 0.9047619047619048

Adei ennada short film la irunthu suturukinga | suturukinga | suturukinga | 0.9565217391304348

Super dialogue  Oruthar mela visvasam kattrathukkaga innoruthavarai asinga paduthathinga |  asing paduthathing | asinga paduthathinga | 0.9

Epadiya jathi padam. Ponnu padama edungada. India vallarasu agidun. Pogada nigalun unga jathi padam. Anda parambarai penda parambarai nu vadhuduraga da. | Ponnupadamaedungada.nigalunungajathipadam. |  Pogada nigalun unga jathi padam. Anda parambarai penda parambarai nu vadhuduraga da. | 0.3464566929133858

Ponnu mella kaivaikuravan Kai mattum illa uyirayim edukanum nanba | Kaimattumillauyirayimedukanum | uyirayim edukanum nanba | 0.6037735849056604

Keka bekka short film bitu copy pa 1:21 enada apditye andha short film ah eduthu 